#### References

1.https://mlexplained.com/2018/09/14/paper-dissected-visualizing-data-using-t-sne-explained/

2.https://www.kaggle.com/maksimeren/covid-19-literature-clustering/

3.https://www.kaggle.com/headsortails/explorations-of-action-moa-eda/

4.https://www.kaggle.com/cdeotte/support-vector-machine-0-925

In this kernel,I have done a basic exploratory data analysis of the MOA datasets.In most of the sections,I have borrowed ideas from [Head's or Tails EDA](https://www.kaggle.com/headsortails/explorations-of-action-moa-eda) kernel and tried to reproduce the same in python.

#### Whats different ?
* Tried to deep dive into the target column names.
* Done clustering,t-SNE of gene expressions to identify if there is any pattern.

If there is any mistake in any of my interpretation,pls let me know in the comments section.Thanks.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from scipy import stats
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv("../input/lish-moa/train_targets_scored.csv")
df_train_nonscored=pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
train_features=pd.read_csv("../input/lish-moa/train_features.csv")
test_features=pd.read_csv("../input/lish-moa/test_features.csv")

In [ ]:
df_train.shape,df_train_nonscored.shape,train_features.shape,test_features.shape

In [ ]:
train_features.head()

Here g-* signifies gene expression data and c-* signifies cell viability.

gene expression - Gene expression is the process by which the information encoded in a gene is used to direct the assembly of a protein molecule. The cell reads the sequence of the gene in groups of three bases.

cell viability- Cell viability is a measure of the proportion of live, healthy cells within a population. Cell viability assays are used to determine the overall health of cells, optimize culture or experimental conditions, and to measure cell survival following treatment with compounds, such as during a drug screen.


In [ ]:
train_features['cp_type'].value_counts()

trt_cp -Compound treatment - A compound treatment study design type is where the response to administration of a compound or chemical (including biological compounds such as hormones) is assayed.

ctl_vehicle - control perturbation . This does not have MOA's.

In [ ]:
train_features['cp_time'].value_counts()

There are three treatment durations - 24 hrs,48 hrs and 72 hrs.

In [ ]:
train_features['cp_dose'].value_counts()

In [ ]:
## Map the categorical features,

train_features['cp_type']=train_features['cp_type'].map({'trt_cp':'0','ctl_vehicle':'1'})
train_features['cp_time']=train_features['cp_time'].map({48:'0',72:'1',24:'2'})
train_features['cp_dose']=train_features['cp_dose'].map({'D1':'0','D2':'1'})

This is the dosage of the drug.-High or low.It is not very clear as to which one is high or which one is low.

In [ ]:
gene_expressions=train_features.columns.str.startswith('g-')
sum(gene_expressions)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
sns.distplot(train_features.loc[:,gene_expressions].median())
plt.title("Distribution of Gene Expression - Plot of Median",fontsize=16)
plt.xlabel("Median of gene expression",fontsize=8)
plt.subplot(2,2,2)
sns.distplot(train_features.loc[:,gene_expressions].max())
plt.title("Distribution of Gene Expression - Plot of Max",fontsize=16)
plt.xlabel("Max of gene expression",fontsize=8)
plt.subplot(2,2,3)
sns.distplot(train_features.loc[:,gene_expressions].min())
plt.title("Distribution of Gene Expression - Plot of Min",fontsize=16)
plt.xlabel("Min of gene expression",fontsize=8)
plt.subplot(2,2,4)
sns.distplot(train_features.loc[:,gene_expressions].std())
plt.title("Distribution of Gene Expression - Plot of std",fontsize=16)
plt.xlabel("Std of gene expression",fontsize=8)

In [ ]:
##Reference -Chris Delotte's kernel.
plt.figure(figsize=(15,15))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_features.iloc[:,i+4])
    plt.title(train_features.columns[i+4])
    plt.xlabel('')

plt.subplots_adjust(hspace=0.4)
plt.show()



The range of gene expression for each feature is different.It appears that the gene expressions g-0,g-3,g-4 are right tailed.

In [ ]:
train_dur_48=train_features[train_features['cp_time']=='0']
fig=plt.figure(figsize=(10,10))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_dur_48.iloc[:,i+4])
    plt.title(train_dur_48.columns[i+4])
fig.suptitle("For treatment duration 48 hrs Gene expression features")
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()

In [ ]:
train_dur_72=train_features[train_features['cp_time']=='1']
fig=plt.figure(figsize=(10,10))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_dur_72.iloc[:,i+4])
    plt.title(train_dur_72.columns[i+4])
fig.suptitle("For treatment duration 72 hrs gene expression features")
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()

In [ ]:
train_dur_24=train_features[train_features['cp_time']=='2']
fig=plt.figure(figsize=(10,10))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_dur_24.iloc[:,i+4])
    plt.title(train_dur_24.columns[i+4])
fig.suptitle("For treatment duration 24 hrs gene expression features")
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()

g-1,g-2 & g-6 range decreases during durations 72 & 24 hrs.

In [ ]:
cell_viability=train_features.columns.str.startswith('c-')
sum(cell_viability)

772 gene expression features and 100 cell viability features.

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,2,1)
sns.distplot(train_features.loc[:,cell_viability].median())
plt.title("Distribution of Cell Viability - Plot of Median",fontsize=16)
plt.xlabel("Median of Cell Viability",fontsize=8)
plt.subplot(2,2,2)
sns.distplot(train_features.loc[:,cell_viability].max())
plt.title("Distribution of Cell Viability - Plot of Max",fontsize=16)
plt.xlabel("Max of Cell Viability",fontsize=8)
#plt.subplot(2,2,3)
# sns.distplot(train_features.loc[:,cell_viability].min())
# plt.title("Distribution of Cell Viability - Plot of Min",fontsize=16)
# plt.xlabel("Min of Cell Viability",fontsize=8)
plt.subplot(2,2,3)
sns.distplot(train_features.loc[:,cell_viability].std())
plt.title("Distribution of Cell Viability - Plot of std",fontsize=16)
plt.xlabel("Std of Cell Viability",fontsize=8)

In [ ]:
train_features.loc[:,cell_viability].min().value_counts()

* Cell viability min value is around -10  for almost all of the cell viability columns.
* Max values are between 3 to 4.It is multimodal.
* Standard deviation peaks at 2.
* From the above stats,we can understand that each individual cell viabilities if plotted could form a bell shaped curve.Lets take few random cell viability columns and plot the same.

In [ ]:
##Reference -Chris Delotte's kernel.
plt.figure(figsize=(15,15))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_features.iloc[:,i+776])
    plt.title(train_features.columns[i+776])

plt.subplots_adjust(hspace=0.4)
plt.show()

The cell viability has a left tailed distribution.Except for c7,the plot shows a peak near -10 for all the plotted viability columns.

Lets plot it for different treatment duration and check their distribution.

In [ ]:
fig=plt.figure(figsize=(10,10))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_dur_48.iloc[:,i+776])
    plt.title(train_dur_48.columns[i+776])
fig.suptitle("For treatment duration 48 hrs Cell Viability features")
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_dur_72.iloc[:,i+776])
    plt.title(train_dur_72.columns[i+776])
fig.suptitle("For treatment duration 72 hrs Cell Viability features")
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))
plt.subplot(3,3,9)
for i in range(9):
    plt.subplot(3,3,i+1)
    sns.distplot(train_dur_24.iloc[:,i+776])
    plt.title(train_dur_24.columns[i+776])
fig.suptitle("For treatment duration 24 hrs Cell Viability features")
fig.tight_layout()
fig.subplots_adjust(top=0.88)
plt.show()

Observing the plots of the duration for the first 8 cell viability features,it is seen that though there is not much difference between duraations 48 vs 72 ,for 24 hours ,the peak at around -10 is missing except for c-6.

In [ ]:
feats=[col for col in train_features.columns if col not in ['sig_id']]
len(feats)

In [ ]:
##Correlation between the features:
#Reference https://www.kaggle.com/senkin13/eda-starter#Feature-Correlation
correlations=train_features[feats].corr().abs().unstack().sort_values(kind='quicksort').reset_index()
correlations=correlations[correlations['level_0']!=correlations['level_1']]
correlations.head(10)

In [ ]:
correlations.tail(10)

### Exploring the class names 

In [ ]:
df_train.head()

This data provides the class labels we need to predict.Lets check for which target classes are there most active MOA's.Before that lets inspect the column names based on the input from [Andrea Zaliani](https://www.kaggle.com/headsortails/explorations-of-action-moa-eda/comments)'s comments in Head's or Tails kernel.

Class names can be receptors or enzymes both are proteins.
 
* Receptors - Targets lying on the surface.They can be modulated positively by agonists and negatively by antagonists.

* Enzymes - Targets lying within the cells.They can be modulated positively by activators and negatively by inhibitors or blockers.

* Agents - Modulators.


In [ ]:
target_sums=df_train.iloc[:,1:].sum(axis=0)

Agent class names,

In [ ]:
## Agents class names,
print('Total Agent classes',len(target_sums.loc[target_sums.index.str.contains('_agent')]))
print('Agent class names & total active MOA for each class')
print(target_sums.loc[target_sums.index.str.contains('_agent')])

Receptor class names,

In [ ]:
## Receptors class names,
print('Total Receptors',len(target_sums.loc[target_sums.index.str.contains('_agonist')])+len(target_sums.loc[target_sums.index.str.contains('_antagonist')]))
# print('Agonist class names & total active MOA for each class')
# print(target_sums.loc[target_sums.index.str.contains('_agonist')])
# print('Antagonist class names & total active MOA for each class')
# print(target_sums.loc[target_sums.index.str.contains('_antagonist')])

In [ ]:
## Enzymes class names,
print('Total Enzymes class names',len(target_sums.loc[target_sums.index.str.contains('_inhibitor')])+len(target_sums.loc[target_sums.index.str.contains('_activator')])+len(target_sums.loc[target_sums.index.str.contains('_blocker')]))

In [ ]:
# print('Inhibitor class names & total active MOA for each class')
# print(target_sums.loc[target_sums.index.str.contains('_inhibitor')])
# print('Activator class names & total active MOA for each class')
# print(target_sums.loc[target_sums.index.str.contains('_activator')])
# print('Blocker class names & total active MOA for each class')
# print(target_sums.loc[target_sums.index.str.contains('_blocker')])

In [ ]:
#Other class names,
print("Total Other class names",len(target_sums.loc[~((target_sums.index.str.contains('_agent')) | 
                  (target_sums.index.str.contains('_inhibitor')) |
                  (target_sums.index.str.contains('_agonist')) | 
                  (target_sums.index.str.contains('_antagonist') | 
                   (target_sums.index.str.contains('_activator') |
                    (target_sums.index.str.contains('_blocker')))))]))
print(target_sums.loc[~((target_sums.index.str.contains('_agent')) | 
                  (target_sums.index.str.contains('_inhibitor')) |
                  (target_sums.index.str.contains('_agonist')) | 
                  (target_sums.index.str.contains('_antagonist') | 
                   (target_sums.index.str.contains('_activator') |
                    (target_sums.index.str.contains('_blocker')))))])

In [ ]:
print(f'''Percentage of inhibitors present {(target_sums[target_sums.index.str.contains('_inhibitor')].count()/206)*100:.2f} %''')
print(f'''Percentage of antagonist present {(target_sums[target_sums.index.str.contains('_antagonist')].count()/206)*100:.2f} %''')
print(f'''Percentage of agonist present {(target_sums[target_sums.index.str.contains('_agonist')].count()/206)*100:.2f} %''')

In [ ]:
#Counter([i[-1] for i in target_sums.index.str.split("_")])

Now that we have checked on the class names,lets see which classes have most and least active MOA.

In [ ]:
df_train.loc[:,target_sums.sort_values(ascending=False).head(5).index].sum(axis=0)

In [ ]:
df_train.loc[:,target_sums.sort_values(ascending=False).tail(5).index].sum(axis=0)

In [ ]:
df_train.loc[:,target_sums.sort_values(ascending=False).tail(10).index].sum(axis=0)

There is a small difference between my results and Head's or Tails.A closer look at this will reveal that there are many target classes with activations of 6 and hence the classes in Head's or Tails kernel match with my results.

It is seen from the top and bottom frequency target classes that most of them are inhibitor,agonist,antagonist.Lets do a quick check of what percentage of these class names are common.

Lets check how many target classes are active at once.

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(df_train.iloc[:,1:].sum(axis=1))
plt.title("Distribution of Multi-labels in target",fontsize=16)
plt.xlabel("Label Count",fontsize=8)

In [ ]:
target_stat=df_train.iloc[:,1:].sum(axis=1).reset_index(drop=True)
(target_stat.value_counts()/target_stat.shape[0])*100

In [ ]:
target_stat.value_counts()

39 % of the sig_ids do not have any MOA's whereas 52 % have 1 MOA.Less than 0.5 % of the sig_ids have more than 4 MOA's.

In [ ]:
single_moa=df_train.loc[(df_train.iloc[:,1:].sum(axis=1))==1]

In [ ]:
(single_moa.iloc[:,1:].idxmax(axis=1).value_counts()/12532)*100

In [ ]:
len(df_train.loc[(df_train.iloc[:,1:].sum(axis=1))==1])

### Non scored train dataset

In [ ]:
df_train_nonscored.head()

In [ ]:
plt.figure(figsize=(8,8))
sns.countplot(df_train_nonscored.iloc[:,1:].sum(axis=1))
plt.title("Distribution of Multi-labels in target(in Non Scored Data)",fontsize=16)
plt.xlabel("Label Count",fontsize=8)

In [ ]:
target_nonsco_stat=df_train_nonscored.iloc[:,1:].sum(axis=1).reset_index(drop=True)
(target_nonsco_stat.value_counts()/target_nonsco_stat.shape[0])*100

In the non-scored dataset,we have 80 % of sig-ids which are not scored whereas 18 % have one MOA.Unlike the target dataset,this one is more sparse.

Lets check whether the sig_ids present are different from the train dataset or we have a case where they are same.

In [ ]:
set(df_train_nonscored['sig_id'])-set(df_train['sig_id'])

The sig_id is same between nonscored and scored dataset.Lets check the columns.

In [ ]:
len(set(df_train_nonscored.columns)-set(df_train.columns))

All 402 target classes present in the non-scored data is different.

In [ ]:
target_sum_nonscored=df_train_nonscored.iloc[:,1:].sum(axis=0)

In [ ]:
set([i[-1] for i in  target_sum_nonscored.index.str.split('_')])

In the comments section of Heads or Tails kernel,Andreas suggests to cluster the gene expression data to check whether they are able distinguish between the target classes - agonist/enzymes and agents.Lets attempt to do and check the results.

In [ ]:
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

In [ ]:
gene_expression=train_features.loc[:,train_features.columns.str.contains('g-')]

In [ ]:
# distortions = []
# K = range(2, 50)
# for k in K:
#     k_means = KMeans(n_clusters=k, random_state=42).fit(gene_expression)
#     k_means.fit(gene_expression)
#     distortions.append(sum(np.min(cdist(gene_expression, k_means.cluster_centers_, 'euclidean'), axis=1)) / gene_expression.shape[0])

In [ ]:
# X_line = [K[0], K[-1]]
# Y_line = [distortions[0], distortions[-1]]

# # Plot the elbow
# plt.plot(K, distortions, 'b-')
# plt.plot(X_line, Y_line, 'r')
# plt.xlabel('k')
# plt.ylabel('Distortion')
# plt.title('The Elbow Method showing the optimal k')
# plt.show()

Lets choose arbitary cluster value of 9 for the sake of simplicity and run k-means.

In [ ]:
k=9
k_means=KMeans(n_clusters=k,random_state=42)
clust=k_means.fit_transform(gene_expression)

In [ ]:
train_feat_cluster=train_features.copy()
train_feat_cluster['cluster']=k_means.labels_

In [ ]:
train_feat_cluster['cluster'].value_counts()

For the purpose of visualizing the clusters,let us apply t-SNE and check the plot.

In [ ]:
## Visualizing the clusters:
tsne=TSNE(n_components=2,verbose=1,perplexity=100,random_state=42)
train_feat_embedded=tsne.fit_transform(gene_expression.values)

In [ ]:
plt.figure(figsize=(10,10))
palette=sns.hls_palette(9,l=0.5,s=0.6)
sns.scatterplot(train_feat_embedded[:,0],train_feat_embedded[:,1],hue=train_feat_cluster['cluster'],palette=palette)
plt.title("t-SNE with cluster",fontsize=10)
plt.xlabel('x',fontsize=8)
plt.ylabel('y',fontsize=8)


Here are my interpretations,
* There is a non-linear separation boundary between each clusters.K-means and t-SNE seem to agree on the points used for separation though there are a few points where the clusters are different.
* Cluster 2 & 3 form a large portion of the gene expression data.



#### To do:
* The above cluster info could be used as a feature in our original dataset along with the other features and see if there is any improvement in scores.

* Use the clusters to check if we are able to differentiate between the class names as mentioned by [Andrea Zaliani](https://www.kaggle.com/headsortails/explorations-of-action-moa-eda/comments).

**Work in progress**